
# Введение #

Линейная регрессия отлично экстраполирует тренды, но не умеет учить взаимодействия. XGBoost отлично учит взаимодействия, но не умеет экстраполировать тренды. В этом уроке мы научимся создавать «гибридные» прогнозировщики, которые комбинируют дополняющие друг друга алгоритмы обучения и позволяют сильным сторонам одного компенсировать слабости другого.

# Компоненты и остатки #

Чтобы проектировать эффективные гибриды, нам нужно лучше понимать, как устроены временные ряды. До сих пор мы изучили три паттерна зависимости: тренд, сезонность и циклы. Многие временные ряды можно хорошо описать аддитивной моделью из этих трёх компонентов плюс некоторой по сути непредсказуемой, полностью случайной *ошибки*:

```
series = trend + seasons + cycles + error
```

Каждый из терминов в этой модели мы называем **компонентом** временного ряда.

**Остатки** модели — это разница между целевой переменной, на которой модель обучалась, и её предсказаниями — другими словами, разница между фактической кривой и аппроксимированной. Если построить остатки против признака, получится «остаточная» часть цели — то, чему модель не научилась по этому признаку.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/mIeeaBD.png" width=700, alt="">
<figcaption style="textalign: center; font-style: italic"><center>Разница между целевым рядом и предсказаниями (синим) даёт ряд остатков.
</center></figcaption>
</figure>

Слева на рисунке выше показан фрагмент ряда *Tunnel Traffic* и кривая тренда‑сезонности из Урока 3. Вычитание аппроксимированной кривой оставляет остатки — справа. Остатки содержат всё из *Tunnel Traffic*, чему тренд‑сезонная модель не научилась.

Можно представить обучение компонент временного ряда как итеративный процесс: сначала выучить тренд и вычесть его из ряда, затем выучить сезонность на детрендированных остатках и вычесть сезоны, затем выучить циклы и вычесть циклы, и, наконец, остаётся только непредсказуемая ошибка.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/XGJuheO.png" width=700, alt="">
<figcaption style="textalign: center; font-style: italic"><center>Пошаговое обучение компонент ряда <em>Mauna Loa CO2</em>. Вычтите аппроксимированную кривую (синим) из ряда, чтобы получить ряд для следующего шага.
</center></figcaption>
</figure>

Сложив все выученные компоненты, мы получим полную модель. По сути, именно это делает линейная регрессия, если обучить её на полном наборе признаков, моделирующих тренд, сезоны и циклы.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/HZEhuHF.png" width=600, alt="">
<figcaption style="textalign: center; font-style: italic"><center>Сложите выученные компоненты и получите полную модель.
</center></figcaption>
</figure>

# Гибридное прогнозирование с остатками #

В предыдущих уроках мы использовали один алгоритм (линейную регрессию), чтобы выучить все компоненты сразу. Но возможно использовать один алгоритм для части компонентов и другой — для остальных. Так можно выбирать лучший алгоритм для каждой компоненты. Для этого один алгоритм обучают на исходном ряде, а второй — на ряде остатков.

В деталях процесс такой:
```
# 1. Обучение и предсказание первой моделью
model_1.fit(X_train_1, y_train)
y_pred_1 = model_1.predict(X_train)

# 2. Обучение и предсказание второй моделью на остатках
model_2.fit(X_train_2, y_train - y_pred_1)
y_pred_2 = model_2.predict(X_train_2)

# 3. Сложение для итоговых предсказаний
y_pred = y_pred_1 + y_pred_2
```

Обычно мы будем использовать разные наборы признаков (`X_train_1` и `X_train_2` выше) в зависимости от того, чему мы хотим научить каждую модель. Например, если первой моделью мы обучаем тренд, то для второй модели обычно не нужен признак тренда.

Хотя можно использовать больше двух моделей, на практике это не слишком полезно. Наиболее распространённая стратегия построения гибридов — та, что описана выше: простой (обычно линейный) алгоритм, за которым следует сложный нелинейный обучатель вроде GBDT или глубокой нейросети. Простая модель обычно служит «помощником» для более мощного алгоритма.

### Проектирование гибридов

Существует много способов комбинировать ML‑модели помимо описанного в уроке. Но успешное комбинирование требует понимания того, как работают эти алгоритмы.

В целом регрессионные алгоритмы могут делать прогнозы двумя способами: либо преобразуя *признаки*, либо преобразуя *цель*. Алгоритмы, преобразующие признаки, учат некоторую математическую функцию, которая берёт признаки на вход и затем комбинирует и преобразует их, чтобы получить выход, соответствующий целевым значениям в обучающем наборе. К таким относятся линейная регрессия и нейросети.

Алгоритмы, преобразующие цель, используют признаки, чтобы группировать целевые значения в обучающем наборе, и предсказывают, усредняя значения внутри группы; набор признаков лишь указывает, какую группу усреднять. К таким относятся деревья решений и k‑ближайшие соседи.

Важно следующее: преобразователи признаков обычно могут **экстраполировать** целевые значения за пределы обучающего набора при наличии подходящих признаков, но прогнозы преобразователей цели всегда ограничены диапазоном обучающего набора. Если временной «дамми‑признак» продолжает считать шаги времени, линейная регрессия продолжает рисовать линию тренда. При том же временном признаке дерево решений будет предсказывать тренд, заданный последним шагом обучающих данных, вечно в будущее. *Деревья решений не умеют экстраполировать тренды.* Случайные леса и градиентный бустинг над деревьями (например XGBoost) — это ансамбли деревьев решений, так что они тоже не умеют экстраполировать тренды.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/ZZtfuFJ.png" width=600, alt="">
<figcaption style="textalign: center; font-style: italic"><center>Дерево решений не сможет экстраполировать тренд за пределы обучающего набора.
</center></figcaption>
</figure>

Эта разница и мотивирует гибрид в этом уроке: использовать линейную регрессию для экстраполяции тренда, преобразовать *цель*, чтобы удалить тренд, и применить XGBoost к детрендированным остаткам. Чтобы «гибридизировать» нейросеть (преобразователь признаков), можно, наоборот, добавить предсказания другой модели как признак — нейросеть включит их в свои предсказания. Метод обучения на остатках — это тот же метод, который использует алгоритм градиентного бустинга, поэтому такие гибриды мы будем называть **бустинговыми**; метод использования предсказаний как признаков называется «стэкингом», поэтому такие гибриды мы будем называть **стэкинговыми**.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
<strong>Победные гибриды из соревнований Kaggle</strong>
    <p>Для вдохновения — несколько лучших решений из прошлых конкурсов:</p>
<ul>
    <li><a href="https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/discussion/8125">STL, бустинг поверх экспоненциального сглаживания</a> — Walmart Recruiting — Store Sales Forecasting</li>
    <li><a href="https://www.kaggle.com/c/rossmann-store-sales/discussion/17896">ARIMA и экспоненциальное сглаживание, бустинг поверх GBDT</a> — Rossmann Store Sales</li> 
    <li><a href="https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/39395">Ансамбль стэкинговых и бустинговых гибридов</a> — Web Traffic Time Series Forecasting</li>
    <li><a href="https://github.com/Mcompetitions/M4-methods/blob/slaweks_ES-RNN/118%20-%20slaweks17/ES_RNN_SlawekSmyl.pdf">Экспоненциальное сглаживание, стэкинг с LSTM‑нейросетью</a> — M4 (не Kaggle)</li>
</ul>
</blockquote>

# Пример — US Retail Sales #

Датасет [*US Retail Sales*](https://www.census.gov/retail/index.html) содержит месячные данные продаж по различным розничным индустриям США за 1992–2019 годы, собранные Бюро переписи США. Наша цель — прогнозировать продажи в 2016–2019 годах по данным предыдущих лет. Помимо гибрида «линейная регрессия + XGBoost», мы также посмотрим, как подготовить датасет временного ряда для использования с XGBoost.

In [ ]:

from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from xgboost import XGBRegressor


simplefilter("ignore")

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)

data_dir = Path("../input/ts-course-data/")
industries = ["BuildingMaterials", "FoodAndBeverage"]
retail = pd.read_csv(
    data_dir / "us-retail-sales.csv",
    usecols=['Month'] + industries,
    parse_dates=['Month'],
    index_col='Month',
).to_period('D').reindex(columns=industries)
retail = pd.concat({'Sales': retail}, names=[None, 'Industries'], axis=1)

retail.head()

Сначала используем модель линейной регрессии, чтобы выучить тренд в каждом ряду. Для демонстрации возьмём квадратичный тренд (порядок 2). (Код здесь по сути такой же, как в предыдущих уроках.) Хотя аппроксимация не идеальна, этого будет достаточно для наших целей.

In [ ]:

y = retail.copy()

# Create trend features
dp = DeterministicProcess(
    index=y.index,  # dates from the training data
    constant=True,  # the intercept
    order=2,        # quadratic trend
    drop=True,      # drop terms to avoid collinearity
)
X = dp.in_sample()  # features for the training data

# Test on the years 2016-2019. It will be easier for us later if we
# split the date index instead of the dataframe directly.
idx_train, idx_test = train_test_split(
    y.index, test_size=12 * 4, shuffle=False,
)
X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
y_train, y_test = y.loc[idx_train], y.loc[idx_test]

# Fit trend model
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

# Make predictions
y_fit = pd.DataFrame(
    model.predict(X_train),
    index=y_train.index,
    columns=y_train.columns,
)
y_pred = pd.DataFrame(
    model.predict(X_test),
    index=y_test.index,
    columns=y_test.columns,
)

# Plot
axs = y_train.plot(color='0.25', subplots=True, sharex=True)
axs = y_test.plot(color='0.25', subplots=True, sharex=True, ax=axs)
axs = y_fit.plot(color='C0', subplots=True, sharex=True, ax=axs)
axs = y_pred.plot(color='C3', subplots=True, sharex=True, ax=axs)
for ax in axs: ax.legend([])
_ = plt.suptitle("Trends")

Хотя алгоритм линейной регрессии способен на многовыходную регрессию, алгоритм XGBoost — нет. Чтобы предсказывать несколько рядов одновременно с помощью XGBoost, мы преобразуем эти ряды из *широкого* формата (по одному ряду в колонке) в *длинный* формат (ряды индексируются категориями по строкам).

In [ ]:
# The `stack` method converts column labels to row labels, pivoting from wide format to long
X = retail.stack()  # pivot dataset wide to long
display(X.head())
y = X.pop('Sales')  # grab target series

Чтобы XGBoost мог отличать наши два временных ряда, мы превратим метки строк для `'Industries'` в категориальный признак с кодированием меток. Также создадим признак годовой сезонности, взяв номера месяцев из временного индекса.

In [ ]:
# Turn row labels into categorical feature columns with a label encoding
X = X.reset_index('Industries')
# Label encoding for 'Industries' feature
for colname in X.select_dtypes(["object", "category"]):
    X[colname], _ = X[colname].factorize()

# Label encoding for annual seasonality
X["Month"] = X.index.month  # values are 1, 2, ..., 12

# Create splits
X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
y_train, y_test = y.loc[idx_train], y.loc[idx_test]

Теперь преобразуем предсказания тренда, полученные ранее, в длинный формат и вычтем их из исходных рядов. Так мы получим детрендированные (остаточные) ряды, которые XGBoost сможет выучить.

In [ ]:
# Pivot wide to long (stack) and convert DataFrame to Series (squeeze)
y_fit = y_fit.stack().squeeze()    # trend from training set
y_pred = y_pred.stack().squeeze()  # trend from test set

# Create residuals (the collection of detrended series) from the training set
y_resid = y_train - y_fit

# Train XGBoost on the residuals
xgb = XGBRegressor()
xgb.fit(X_train, y_resid)

# Add the predicted residuals onto the predicted trends
y_fit_boosted = xgb.predict(X_train) + y_fit
y_pred_boosted = xgb.predict(X_test) + y_pred

Аппроксимация выглядит довольно хорошо, хотя видно, что тренд, выученный XGBoost, хорош ровно настолько, насколько хорош тренд, выученный линейной регрессией — в частности, XGBoost не смог компенсировать плохо подогнанный тренд в ряду `'BuildingMaterials'`.

In [ ]:

axs = y_train.unstack(['Industries']).plot(
    color='0.25', figsize=(11, 5), subplots=True, sharex=True,
    title=['BuildingMaterials', 'FoodAndBeverage'],
)
axs = y_test.unstack(['Industries']).plot(
    color='0.25', subplots=True, sharex=True, ax=axs,
)
axs = y_fit_boosted.unstack(['Industries']).plot(
    color='C0', subplots=True, sharex=True, ax=axs,
)
axs = y_pred_boosted.unstack(['Industries']).plot(
    color='C3', subplots=True, sharex=True, ax=axs,
)
for ax in axs: ax.legend([])

# Ваш ход #

[**Прогнозируйте Store Sales**](https://www.kaggle.com/kernels/fork/19616007) с XGBoost‑гибридом и попробуйте другие комбинации ML‑алгоритмов.

---




*Есть вопросы или комментарии? Посетите [форум обсуждений курса](https://www.kaggle.com/learn/time-series/discussion), чтобы пообщаться с другими учащимися.*